In [1]:
import os
import json
import importlib
import numpy as np
import torch
import tqdm

import phaselocknet_model
import util

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


device(type='cuda', index=0)

In [32]:
importlib.reload(phaselocknet_model)
importlib.reload(util)

dir_model = "../phaselocknet/models/sound_localization/simplified_IHC3000_delayed_integration/arch01"
input_shape = [2, 60000, 2]
config_random_slice = {"size": [50, 10000], "buffer": [0, 500]}

dir_model = "../phaselocknet/models/sound_localization/IHC3000_delayed_integration/arch01"
input_shape = [2, 3, 50, 12000, 2]
config_random_slice = {"size": [50, 10000], "buffer": [0, 500]}

# dir_model = "../phaselocknet/models/spkr_word_recognition/simplified_IHC3000/arch0_0000"
# input_shape = [2, 40000]
# config_random_slice = {"size": [50, 20000], "buffer": [0, 0]}

# dir_model = "../phaselocknet/models/spkr_word_recognition/IHC3000/arch0_0000"
# input_shape = [2, 3, 50, 20000]
# config_random_slice = {}


with open(os.path.join(dir_model, "config.json")) as f:
    config_model = json.load(f)
with open(os.path.join(dir_model, "arch.json")) as f:
    architecture = json.load(f)

model = phaselocknet_model.Model(
    config_model=config_model,
    architecture=architecture,
    input_shape=input_shape,
    config_random_slice=config_random_slice,
).to(device)

for k, v in model(torch.zeros(model.input_shape, device=device)).items():
    print(k, v.shape, v.dtype)


label_loc_int torch.Size([2, 504]) torch.float32


In [33]:
for _ in tqdm.tqdm(range(200)):
    x = torch.randn(model.input_shape, device=device)
    model(x)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 85.26it/s]


In [35]:
importlib.reload(util)

util.load_tf_model_checkpoint(
    model=model.perceptual_model,
    filename=os.path.join(dir_model, "ckpt_BEST"),
)


[load_tf_model_checkpoint] ../phaselocknet/models/sound_localization/IHC3000_delayed_integration/arch01/ckpt_BEST


'../phaselocknet/models/sound_localization/IHC3000_delayed_integration/arch01/ckpt_BEST'

In [36]:
for k, v in model.perceptual_model.state_dict().items():
    print(k, v.shape)
# len(model.perceptual_model.state_dict())


body.conv_0.weight torch.Size([32, 3, 1, 8])
body.conv_0.bias torch.Size([32])
body.batch_norm_0.weight torch.Size([32])
body.batch_norm_0.bias torch.Size([32])
body.batch_norm_0.running_mean torch.Size([32])
body.batch_norm_0.running_var torch.Size([32])
body.batch_norm_0.num_batches_tracked torch.Size([])
body.conv_1.weight torch.Size([32, 16, 1, 64])
body.conv_1.bias torch.Size([32])
body.batch_norm_1.weight torch.Size([32])
body.batch_norm_1.bias torch.Size([32])
body.batch_norm_1.running_mean torch.Size([32])
body.batch_norm_1.running_var torch.Size([32])
body.batch_norm_1.num_batches_tracked torch.Size([])
body.conv_2.weight torch.Size([32, 16, 1, 64])
body.conv_2.bias torch.Size([32])
body.batch_norm_2.weight torch.Size([32])
body.batch_norm_2.bias torch.Size([32])
body.batch_norm_2.running_mean torch.Size([32])
body.batch_norm_2.running_var torch.Size([32])
body.batch_norm_2.num_batches_tracked torch.Size([])
body.conv_3.weight torch.Size([64, 32, 2, 4])
body.conv_3.bias torch.